# Building LLM applications: Notebook 03

# Tools

## Initialize

In [1]:
import os
import dotenv
import json

from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool

from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser

from langchain_ollama import ChatOllama

In [2]:
MODEL = 'llama3.2:3b-instruct-fp16'

In [ ]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

## Calling tools manually

### Step 1: Creat tools

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    print(f"Adding {a} + {b}")
    return a + b


@tool
def sub(a: int, b: int) -> int:
    """Substract two numbers"""
    print(f"Substracting {a} - {b}")
    return a - b


tools = [add, sub]

tools_map = {t.name.lower(): t for t in tools}
tools_map

In [14]:
def invoke_tool(llm_response, tools_map):
    """
    Call the tools, resurn a list of "tools results""
    """
    tool_msgs = []
    for tool_call in llm_response.tool_calls:
        tool_name = tool_call['name'].lower()
        tool = tools_map.get(tool_name)
        if tool:
            tool_msg = tool.invoke(tool_call)
            print(tool_msg)
        print(f"Tool result: {tool_msg}")
        tool_msgs.append(tool_msg)
    return tool_msgs

### Step 3: Bind tools to the LLM, invoke it

In [ ]:
query = "What is the addition of 40 and 2?"
message = HumanMessage(query)
messages = [message]                # The list of messages to send to the LLM, now is only one message


In [13]:
# ADD YOUR CODE HERE
# Create a runnable to send the messages to the LLM
#
#

### Step 4: Call the tools (as specifief in the LLM response)

In [11]:
# ADD CODE HERE
# Invoke the tool/s and add the 'tool results' to the messages list
# You can use the function invoke_tool(..)
#

### Step 5: Append the tool's output, invoke the LLM again

In [15]:
# ADD YOUR CODE HERE
# Invoke the LLM with the messages (including the tool results)
#
#